In [33]:
!git clone https://github.com/jakob-beetz/sbe-2025-lca-workshop.git


fatal: destination path 'sbe-2025-lca-workshop' already exists and is not an empty directory.


In [34]:
!pip install ifcopenshell

In [35]:
# prompt: i want to clone a single script into here which is located at https://github.com/IfcOpenShell/IfcOpenShell/raw/refs/heads/v0.8.0/src/ifccsv/ifccsv.py

!curl https://raw.githubusercontent.com/IfcOpenShell/IfcOpenShell/refs/heads/v0.8.0/src/ifccsv/ifccsv.py -o ifccsv.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22371  100 22371    0     0  69846      0 --:--:-- --:--:-- --:--:-- 69909


In [36]:
import ifcopenshell

In [37]:
m = ifcopenshell.open("./sbe-2025-lca-workshop/data/IFC_models/Duplex_QTO.ifc")
m = ifcopenshell.open("/content/sbe-2025-lca-workshop/data/IFC_models/HiLo.ifc")

Let's inspect the Model. Get all the spaces

In [38]:
m.by_type("IfcSpace")

[]

Ok, these are the raw entity instances as they appear in the IFC text (SPFF) file. Not very readable for the non-technical reader.

Let's just list the names:

In [39]:
for space in m.by_type("IfcSpace"):
    print(space.LongName)

In [40]:
for wall in m.by_type("IfcWall"):
    print(wall.Name)

Basic Wall:Wärmedämmung_200:687589
Basic Wall:Wärmedämmung_200:687601
Basic Wall:BSP 140 - 5s:725224
Basic Wall:Wärmedämmung_200:725935
Basic Wall:Wärmedämmung_200:882225
Basic Wall:Holzfassade hinterlüftet:1105047
Basic Wall:Holzfassade hinterlüftet:1434003
Basic Wall:Holzfassade hinterlüftet:1434022
Basic Wall:BSP 140 - 5s:1686981
Basic Wall:BSP 140 - 5s:1687276
Basic Wall:BSP 140 Sichtseite einseitig:1687929
Basic Wall:BSP 140 - 5s:1689984
Basic Wall:BSP 140 - 5s:1691966
Basic Wall:BSP 140 Sichtseite einseitig:1697327
Basic Wall:BSP 140 - 5s:1704351
Basic Wall:Wärmedämmung_Mineralwolle_130:1839774
Basic Wall:Wärmedämmung_Mineralwolle_130:1839949
Basic Wall:BSP 140 - 5s:1873452
Basic Wall:BSP 140 - 5s:1879906
Basic Wall:BSP 140 - 5s:1949758
Basic Wall:BSP 140 - 5s:2854498
Basic Wall:BSP 140 Sichtseite einseitig:2877799
Basic Wall:BSP 140 Sichtseite einseitig:3258052
Basic Wall:3-Schichtplatte 27mm C+/C:3384320
Basic Wall:Wärmedämmung_200:5056218
Basic Wall:Wärmedämmung_200:5247902
Ba

In [41]:
for wall in m.by_type("IfcWall"):
  for rel in wall.HasAssociations:
    if rel.is_a("IfcRelAssociatesMaterial"):
      if rel.RelatingMaterial.is_a("IfcMaterial"):
        print(f"Wall: {wall.Name}, Material: {rel.RelatingMaterial.Name}")
      elif rel.RelatingMaterial.is_a("IfcMaterialList"):
        material_names = [mat.Name for mat in rel.RelatingMaterial.Materials]
        print(f"Wall: {wall.Name}, Materials: {', '.join(material_names)}")
      elif rel.RelatingMaterial.is_a("IfcMaterialLayerSetUsage"):
        material_names = [layer.Material.Name for layer in rel.RelatingMaterial.ForLayerSet.MaterialLayers]
        print(f"Wall: {wall.Name}, Material Layers: {', '.join(material_names)}")
      elif rel.RelatingMaterial.is_a("IfcMaterialConstituentSet"):
          constituents = []
          for constituent in rel.RelatingMaterial.MaterialConstituents:
              if constituent.Material:
                  constituents.append(constituent.Material.Name)
          print(f"Wall: {wall.Name}, Material Constituents: {', '.join(constituents)}")

Wall: Basic Wall:Innenwandelement Archisonic abnehmbar:687694, Material Layers: Archisonic Charcoal, MDF
Wall: Basic Wall:Wärmedämmung_Slentex 70:687731, Material Layers: Wärmedämmung Slentex Aerogel
Wall: Basic Wall:Holzfassade_Hinterlüftet_OhneUK:687806, Material Layers: Holzschalung Vertikal, Nadelholz,  schwarz gestrichen
Wall: Basic Wall:PV Fassade Terasse:687807, Material Layers: Körpermodell
Wall: Basic Wall:Wärmedämmung_200:726297, Material Layers: Wärmedämmung Mineralwolle, 0.035 W/mK, Wärmedämmung Mineralwolle, 0.035 W/mK
Wall: Basic Wall:Innenwandelement Archisonic fix:1400196, Material Layers: Archisonic Charcoal, MDF
Wall: Basic Wall:Holzfassade_Hinterlüftet_OhneUK:1434021, Material Layers: Holzschalung Vertikal, Nadelholz,  schwarz gestrichen
Wall: Basic Wall:Holzfassade hinterlüftet:1434444, Material Layers: Holzschalung Vertikal, Nadelholz,  schwarz gestrichen, Luft, Luft
Wall: Basic Wall:Wärmedämmung_200:1434709, Material Layers: Wärmedämmung Mineralwolle, 0.035 W/mK, 

In [42]:
import ifcopenshell.util.selector
wall = m.by_type("IfcWall")[3]
# Get the Name attribute of the wall's type.
print(ifcopenshell.util.selector.get_element_value(wall, "material.Name"))
print(ifcopenshell.util.selector.get_element_value(wall, "Dimensions.Area"))

Basic Wall:Wärmedämmung_200
17.324505074342


In [43]:
import json, ifcopenshell
from ifcopenshell.util import selector
from ifccsv import IfcCsv

# ---- 0. load IFC & profile --------------------
IFC_PATH   = "./sbe-2025-lca-workshop/data/IFC_models/Duplex_QTO.ifc"
IFC_PATH   = "./sbe-2025-lca-workshop/data/IFC_models/Duplex_QTO.ifc"
PROFILE_JS = "./sbe-2025-lca-workshop/data/all_materials_query_duplex.json"

f = ifcopenshell.open(IFC_PATH)
profile = json.load(open(PROFILE_JS))

# ---- 1. element query -------------------------
query      = profile["query"]
elements   = selector.filter_elements(f, query)

# ---- 2. attributes, headers, formatting -------
attrs      = [a["name"] for a in profile["attributes"]]
headers    = [a["header"] for a in profile["attributes"]]
formatting = [a["formatting"] for a in profile["attributes"]]

# optional features
sort       = [a["sort"]     for a in profile["attributes"]]
# Modified: Extract the full dictionary for each group, not just the 'group' value
groups     = [a["group"] for a in profile["attributes"] if "group" in a]
summaries  = [a["summary"]  for a in profile["attributes"]]

# ---- 3. settings / defaults -------------------
s = profile["settings"]
csv_opts = dict(
    include_global_id = s.get("include_global_id", True),
    delimiter         = s.get("csv_custom_delimiter") or s.get("csv_delimiter", ","),
    null              = s.get("null_value", "N/A"),
    empty             = s.get("empty_value", "-"),
    bool_true         = s.get("true_value", "YES"),
    bool_false        = s.get("false_value", "NO"),
    concat            = s.get("concat_value", ", "),
    should_preserve_existing = s.get("should_preserve_existing", False),
)

fmt = s.get("format", "csv").lower()          # csv / ods / xlsx / pd
out = f"export.{fmt if fmt!='pd' else 'csv'}" # pd returns DataFrame

# ---- 4. export --------------------------------
ic = IfcCsv()
ic.export(
    f,                 # ifcopenshell.file
    elements,
    attributes = attrs,
    headers    = headers,
    output     = out if fmt != "pd" else None,
    format     = fmt,  # IfcCsv.FILE_FORMAT enum resolves str
    **csv_opts
)

if fmt == "pd":
    df = ic.export_pd()          # you get a pandas.DataFrame to post-process
    df.to_csv(out, index=False)

print(f"✓ Exported {len(elements)} rows to {out}")

✓ Exported 295 rows to export.web


In [44]:
df = ic.export_pd()
df

,GlobalId,Material,IFC Class,NetSideArea,type,Area,Volume,mat.0,mat.1,mat.2,mat.3,materials
0,0jf0rYHfX3RAB3bSIRjmxl,Basic Roof:Live Roof over Wood Joist Flat Roof,IfcRoof,N/A,N/A,132.926036,60.747198,Site - Grass,Roofing - Barrier,Roofing - EPDM Membrane,Insulation / Thermal Barriers - Rigid insulation,"#22616=IfcMaterial('Site - Grass'), #22625=Ifc..."
1,2O2Fr$t4X7Zf8NOew3FNhv,Basic Wall:Exterior - Brick on Block,IfcWallStandardCase,48.4321,N/A,48.4321,20.196186,Masonry - Brick,Misc. Air Layers - Air Space,Insulation / Thermal Barriers - Rigid insulation,Masonry - Concrete Block,"#3876=IfcMaterial('Masonry - Brick'), #3893=If..."
2,2O2Fr$t4X7Zf8NOew3FLR9,Basic Wall:Exterior - Brick on Block,IfcWallStandardCase,41.357137,N/A,41.357137,16.741648,Masonry - Brick,Misc. Air Layers - Air Space,Insulation / Thermal Barriers - Rigid insulation,Masonry - Concrete Block,"#3876=IfcMaterial('Masonry - Brick'), #3893=If..."
3,2O2Fr$t4X7Zf8NOew3FNqI,Basic Wall:Exterior - Brick on Block,IfcWallStandardCase,49.7248,N/A,49.7248,20.735242,Masonry - Brick,Misc. Air Layers - Air Space,Insulation / Thermal Barriers - Rigid insulation,Masonry - Concrete Block,"#3876=IfcMaterial('Masonry - Brick'), #3893=If..."
4,0jf0rYHfX3RAB3bSIRjmmy,Basic Wall:Exterior - Brick on Block,IfcWallStandardCase,5.3592,N/A,5.3592,2.234786,Masonry - Brick,Misc. Air Layers - Air Space,Insulation / Thermal Barriers - Rigid insulation,Masonry - Concrete Block,"#3876=IfcMaterial('Masonry - Brick'), #3893=If..."
...,...,...,...,...,...,...,...,...,...,...,...,...
290,0BTBFw6f90Nfh9rP1dl_3A,N/A,IfcSpace,N/A,N/A,26.119314,56.893582,N/A,N/A,N/A,N/A,
291,0iEHWY1$XA8eQeeULq4jQJ,N/A,IfcFurnishingElement,N/A,"[0iEHWY1$XA8eQeeULq4jRb, [[#30=IfcPerson($,$,'...",N/A,N/A,N/A,N/A,N/A,N/A,
292,1hOSvn6df7F8_7GcBWlSnC,N/A,IfcWindow,N/A,"[26vTxWa597WQIUR7EELBWP, [[#30=IfcPerson($,$,'...",N/A,N/A,N/A,N/A,N/A,N/A,
293,1hOSvn6df7F8_7GcBWlS1M,N/A,IfcWindow,N/A,"[3N9NZSp2HBSBX50KQyBDXG, [[#30=IfcPerson($,$,'...",N/A,N/A,N/A,N/A,N/A,N/A,


In [45]:
# Convert 'Area' column to numeric, coercing errors
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')


# Group by 'mat.0' and sum the 'Area'
grouped_df = df.groupby('mat.1')['Area'].sum().reset_index()

# Rename the summed column for clarity (optional)
grouped_df = grouped_df.rename(columns={'Area': 'Total Area'})

# Print or display the result
print("Grouped and summarized Area by mat.0:")
grouped_df

Grouped and summarized Area by mat.0:


,mat.1,Total Area
0,Insulation / Thermal Barriers - Semi-rigid ins...,196.920616
1,Masonry - Grout,44.909772
2,Metal - Stud Layer,177.675640
3,Misc. Air Layers - Air Space,271.999416
4,N/A,1211.175126
5,Plasterboard,304.570873
6,Roofing - Barrier,132.926036
7,Wood - Dimensional Lumber,118.562765
